# G<sub>0</sub>W<sub>0</sub> Approximation Tutorial Notebook

In this notebook, we present an example calculation of quasiparticle energies using QuatumMASALA's `gw` module.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports
from __future__ import annotations
import numpy as np
import sys


sys.path.append(".")

dirname = "../../../tests/bgw/silicon/cohsex/"
# dirname = "../../../tests/bgw/gaas_4/cohsex/"
# dirname = "../gw_old/scripts/results/si_6_nband272/si_6_gw/"

### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [3]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
# epsinp = Epsinp(epsilon_cutoff=1.2,
#                 use_wfn_hdf5=True,
#                 number_bands=8,
#                 write_vcoul=True,
#                 qpts=[[0.0,0.0,0.0]],
#                 is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Load WfnData
Calculation of dielectric matrix requires mean field eigenfunctions. \
Wavefunction data generated from mean-field codes can be read using the ``wfn2py`` utility, which assumes that the incoming data satisfies BerkeleyGW's [`wfn_h5`](http://manual.berkeleygw.org/3.0/wfn_h5_spec/) specification. The data is stored as a `NamedTuple` object.

For reasons discussed later, we also require wavefunctions on a shifted grid to calculate dielectric matrix at $q\to 0$. This shifted grid dataset will be referred to as `wfnqdata`.

Similarly, the utilities `read_rho` and `read_vxc` can be used to read density and exchange-correlation respectively.

In [4]:
# wfn2py
from qtm.gw.io_bgw import inp
from qtm.gw.io_bgw.wfn2py import wfn2py

wfndata = wfn2py(dirname+'WFN.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
# print(wfndata.__doc__)

wfnqdata = wfn2py(dirname+'WFNq.h5')#, wfn_ecutrho_minus_ecutwfn=epsinp.epsilon_cutoff)
# print(wfnqdata.__doc__)

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [5]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

# Manual initialization
# epsilon = Epsilon(
#     crystal = wfndata.crystal,
#     gspace = wfndata.grho,
#     kpts = wfndata.kpts,
#     kptsq = wfnqdata.kpts,
#     l_wfn = wfndata.l_wfn,
#     l_wfnq = wfnqdata.l_wfn,
#     l_gsp_wfn = wfndata.l_gk,
#     l_gsp_wfnq = wfnqdata.l_gk,
#     qpts = QPoints.from_cryst(wfndata.kpts.recilat, epsinp.is_q0, *epsinp.qpts),
#     epsinp = epsinp,
# )

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 16183.48it/s]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix

<!-- 1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}({\textbf k},{\textbf q},{\textbf G}) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$.
    Since this is a convolution in k-space, the time complexity can be reduced from $\mathcal{O}\left(N^2_G\right)$ to $\mathcal{O}\left(N_G\ln N_G\right)$ by using Fast Fourier Transform, where $N_G$  the number of reciprocal lattice vectors in the wavefunction.
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in BGW2012, wherein $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid. -->

In [6]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
            print(epsref[:2,:2])
            print(epsinv[:2,:2])

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

# epsinp.no_min_fftgrid = False
# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
# calculate_epsilon()


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 15645.83it/s]


Epsilon> q-pt index: 100%|██████████| 64/64 [00:24<00:00,  2.57it/s]


### Sigma Calculation

In [7]:
from qtm.gw.sigma import Sigma

outdir = dirname+"temp/"

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
    outdir=outdir,
)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 16146.49it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7f2800dabe50>
        * qpts = <qtm.gw.core.QPoints object at 0x7f27fe561160>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [00:48<00:00,  1.32it/s]


In [8]:
sigma_sx_cohsex_mat = sigma.sigma_sx_static(yielding=True)    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(sigma_sx_cohsex_mat)

(137,)


Sigma_SX_Static:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 29.26it/s]

Sigma SX COHSEX
   11.897822   10.692213   11.350437
    8.050415   10.691906   10.025763
    8.052588    8.736187    8.384990
    8.052483    8.736867    8.385902
    2.917532    2.597576    2.882170
    2.915433    2.597634    2.433213
    2.919747    1.333271    2.431261
    2.674705    1.334266    0.749947


from tqdm.auto import tqdm
sigma_x_mat = sigma.sigma_x(yielding=True)    
print("Sigma X")
sigma.pprint_sigma_mat(sigma_x_mat)

In [9]:
sigma_ch_cohsex_mat = sigma.sigma_ch_static()    
print("Sigma CH COHSEX")
sigma.pprint_sigma_mat(sigma_ch_cohsex_mat)

Sigma_CH_Static_Partial:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:03<00:00, 16.30it/s]

Sigma CH COHSEX
   -6.641277   -6.317908   -6.511362
   -5.113976   -6.317977   -6.076655
   -5.114200   -5.580271   -5.418842
   -5.114077   -5.580427   -5.419831
   -4.749343   -4.710424   -4.933476
   -4.748586   -4.710832   -4.288478
   -4.750098   -3.472540   -4.288100
   -4.781285   -3.473914   -2.996284


In [10]:
sigma.autosave=False
sigma.print_condition=True
cohsex_result = sigma.calculate_static_cohsex()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 30.22it/s]


Sigma X
  -17.544906  -16.031120  -16.916213
  -12.984799  -16.031115  -14.895720
  -12.984990  -13.425014  -13.214186
  -12.984879  -13.425073  -13.214228
   -5.772040   -5.203257   -5.937991
   -5.772092   -5.202819   -5.086562
   -5.772459   -3.793211   -5.086561
   -5.801537   -3.793222   -2.418507
(137,)


Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 29.48it/s]


Sigma SX STATIC
   11.897822   10.692213   11.350437
    8.050415   10.691906   10.025763
    8.052588    8.736187    8.384990
    8.052483    8.736867    8.385902
    2.917532    2.597576    2.882170
    2.915433    2.597634    2.433213
    2.919747    1.333271    2.431261
    2.674705    1.334266    0.749947


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:03<00:00, 16.36it/s]


Sigma CH STATIC
   -6.641277   -6.317908   -6.511362
   -5.113976   -6.317977   -6.076655
   -5.114200   -5.580271   -5.418842
   -5.114077   -5.580427   -5.419831
   -4.749343   -4.710424   -4.933476
   -4.748586   -4.710832   -4.288478
   -4.750098   -3.472540   -4.288100
   -4.781285   -3.473914   -2.996284


Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:23<00:00,  2.74it/s]

Sigma CH EXACT STATIC
   -7.461535   -7.645228   -7.650968
   -7.788499   -7.645261   -7.324929
   -7.788918   -7.500747   -7.703410
   -7.788862   -7.501264   -7.704059
   -7.361444   -6.804707   -7.377943
   -7.360928   -6.804653   -7.142290
   -7.362404   -7.712695   -7.142081
   -7.851026   -7.714351   -6.239520
Sig (Exact):
  -13.108618  -12.984135  -13.216744
  -12.722883  -12.984470  -12.194886
  -12.721321  -12.189574  -12.532606
  -12.721258  -12.189470  -12.532385
  -10.215951   -9.410387  -10.433764
  -10.217588   -9.409837   -9.795639
  -10.215116  -10.172634   -9.797381
  -10.977857  -10.173308   -7.908080
Eqp0 (Exact):
   -8.400645   -3.702053   -5.747930
    4.873471   -3.702388   -2.754254
    4.875034    1.794270    3.591356
    4.875096    1.794374    3.591578
    8.702105    6.618048    7.562137
    8.700468    6.618598    9.542243
    8.702940   16.821542    9.540501
    9.643758   16.820869   13.970027
Sig (Partial):
  -12.288361  -11.656815  -12.077138
  -10.04836

In [11]:
from qtm.gw.io_bgw.sigma_hp_reader import read_sigma_hp
ref_dict = read_sigma_hp(dirname+"sigma_hp.log")
for ik in cohsex_result:
    print("k-point index:",ik)
    qtty = 'Eqp1'
    print(np.abs(ref_dict[ik+1][qtty]-np.around(cohsex_result[ik][qtty],6)))

k-point index: 0
[3.6e-05 2.0e-05 2.4e-05 2.4e-05 5.0e-06 7.0e-06 2.0e-06 2.3e-05]
k-point index: 40
[1.8e-05 2.6e-05 5.0e-06 1.0e-05 1.9e-05 1.3e-05 3.9e-05 3.0e-05]
k-point index: 8
[3.9e-05 2.0e-06 1.4e-05 1.5e-05 3.0e-06 5.0e-06 7.0e-06 2.2e-05]


In [12]:
sigma.print_condition=True
sigma_ch_exact_mat = sigma.sigma_ch_static_exact()    
print("Sigma CH COHSEX EXACT")
sigma.pprint_sigma_mat(sigma_ch_exact_mat)

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:23<00:00,  2.75it/s]

Sigma CH COHSEX EXACT
   -7.461535   -7.645228   -7.650968
   -7.788499   -7.645261   -7.324929
   -7.788918   -7.500747   -7.703410
   -7.788862   -7.501264   -7.704059
   -7.361444   -6.804707   -7.377943
   -7.360928   -6.804653   -7.142290
   -7.362404   -7.712695   -7.142081
   -7.851026   -7.714351   -6.239520


In [13]:
sigma.print_condition=False
sigma_ch_gpp,_ = sigma.sigma_ch_gpp()    
print("Sigma CH GPP")
sigma.pprint_sigma_mat(sigma_ch_gpp)

Sigma_CH_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_GPP: 100%|██████████| 64/64 [00:54<00:00,  1.17it/s]

Sigma CH GPP
   -5.809203   -5.862668   -5.896258
   -5.426201   -5.862915   -5.647912
   -5.427718   -5.567567   -5.584283
   -5.416109   -5.568202   -5.586360
   -5.398088   -4.984852   -5.448451
   -5.397232   -4.984614   -4.889763
   -5.400241   -4.661941   -4.885163
   -5.754373   -4.670230   -3.779749


In [14]:
gpp_result = sigma.calculate_gpp()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 30.43it/s]

Sigma X GPP


  -17.544906  -16.031120  -16.916213
  -12.984799  -16.031115  -14.895720
  -12.984990  -13.425014  -13.214186
  -12.984879  -13.425073  -13.214228
   -5.772040   -5.203257   -5.937991
   -5.772092   -5.202819   -5.086562
   -5.772459   -3.793211   -5.086561
   -5.801537   -3.793222   -2.418507


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:03<00:00, 16.43it/s]

...

   -7.461535   -7.645228   -7.650968
   -7.788499   -7.645261   -7.324929
   -7.788918   -7.500747   -7.703410
   -7.788862   -7.501264   -7.704059
   -7.361444   -6.804707   -7.377943
   -7.360928   -6.804653   -7.142290
   -7.362404   -7.712695   -7.142081
   -7.851026   -7.714351   -6.239520
Started sigma_sx_gpp 2023-09-03 21:34:45


Sigma_SX_GPP: 100%|██████████| 64/64 [00:50<00:00,  1.27it/s]

Sigma SX GPP


   12.264463   10.862820   11.567866
    8.203939   10.862420   10.184037
    8.207702    8.852984    8.518146
    8.196115    8.854197    8.520741
    3.356442    2.869535    3.293694
    3.353732    2.869395    2.792918
    3.358630    1.779970    2.786417
    3.373766    1.787217    1.095793


Sigma_CH_GPP:   5%|▍         | 3/64 [00:02<00:49,  1.22it/s]

In [ ]:
from qtm.gw.io_bgw.sigma_hp_reader import read_sigma_hp

ref_dict = read_sigma_hp(dirname+"../gpp/sigma_hp.log")
for ik in gpp_result:
    print("k-point index:",ik)
    qtty = 'Eqp1'
    print(np.abs(ref_dict[ik+1][qtty]-np.around(gpp_result[ik][qtty],6)))

k-point index: 0
[0.000236 0.000262 0.000678 0.004735 0.004289 0.001108 0.001276 0.001741]
k-point index: 40
[1.067e-03 7.200e-05 2.290e-03 5.800e-05 6.000e-05 1.991e-03 8.690e-04
 6.440e-04]
k-point index: 8
[8.630e-04 1.535e-03 2.410e-04 3.362e-03 4.570e-04 1.992e-03 2.456e-03
 9.600e-05]
